# Imports

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, root_mean_squared_error, median_absolute_error, mean_absolute_error
from sklearn.model_selection import train_test_split

In [2]:
import sys
sys.path.append('..')
from utils.estimators import *

In [3]:
pd.options.mode.chained_assignment = None

# Load data

In [4]:
df = pd.read_csv('../processed_data/england_wales.csv')

In [5]:
yob_order = (y := sorted(df['yob'].unique()))[-1:] + y[:-1]

In [6]:
df['yob'] = pd.Categorical(df['yob'], categories=yob_order)

In [7]:
df.sort_values('yob', inplace=True)

In [8]:
religion_cols = sorted(df['religion'].unique())

In [9]:
qwe = df.pivot_table(
    index=['census_year', 'sex', 'yob', 'geo_code'],
    columns='religion',
    values='population',
    aggfunc='sum',
    observed=True,
)

# Holdout a test set

In [10]:
np.random.seed(808)
geo_codes = df['geo_code'].unique()
test_geo_codes = pd.Series(geo_codes).sample(len(geo_codes) // 2)

# Build train and validate sets for over tens

## Add a test flag

In [11]:
qwe['test_flag'] = qwe.index.get_level_values('geo_code').isin(test_geo_codes)

In [12]:
qwe['train_X'] = (qwe.index.get_level_values('census_year') == 2001) & (~qwe['test_flag'])
qwe['train_Y'] = (qwe.index.get_level_values('census_year') == 2011) & (~qwe['test_flag'])
qwe['train_Z'] = (qwe.index.get_level_values('census_year') == 2021) & (~qwe['test_flag'])

In [13]:
qwe['test_X'] = (qwe.index.get_level_values('census_year') == 2001) & (qwe['test_flag'])
qwe['test_Y'] = (qwe.index.get_level_values('census_year') == 2011) & (qwe['test_flag'])
qwe['test_Z'] = (qwe.index.get_level_values('census_year') == 2021) & (qwe['test_flag'])

## Add age_as_of columns

### 2001

In [14]:
age_bands_2001 = dict(df.loc[df['census_year'] == 2001, ['yob', 'age_band']].drop_duplicates().values)
age_bands_2001['pre-1927'] = '75+'

qwe['age_2001'] = pd.Categorical(
    qwe.index.get_level_values('yob').map(age_bands_2001),
)

### 2011

In [15]:
age_bands_2011 = dict(df.loc[df['census_year'] == 2011, ['yob', 'age_band']].drop_duplicates().values)
age_bands_2011['pre-1927'] = '75+'
age_bands_2011['1927-1931'] = '75+'
age_bands_2011['1932-1936'] = '75+'

qwe['age_2011'] = pd.Categorical(
    qwe.index.get_level_values('yob').map(age_bands_2011),
    categories=qwe['age_2001'].cat.categories
)

### 2021

In [16]:
age_bands_2021 = dict(df.loc[df['census_year'] == 2021, ['yob', 'age_band']].drop_duplicates().values)
age_bands_2021['pre-1927'] = '85+'
age_bands_2021['1927-1931'] = '85+'
age_bands_2021['1932-1936'] = '85+'

qwe['age_2021'] = pd.Categorical(
    qwe.index.get_level_values('yob').map(age_bands_2021),
    categories=sorted(pd.Series(age_bands_2021.values()).unique())
)

## Build datasets

In [17]:
df_train_X = qwe.loc[qwe['train_X']].groupby(['sex', 'age_2001', 'geo_code'], observed=True)[religion_cols].sum().apply(lambda x:x/x.sum(), axis='columns')
df_train_Y = qwe.loc[qwe['train_Y']].groupby(['sex', 'age_2001', 'geo_code'], observed=True)[religion_cols].sum().apply(lambda x:x/x.sum(), axis='columns')
df_valid_Y = qwe.loc[qwe['train_Y']].groupby(['sex', 'age_2011', 'geo_code'], observed=True)[religion_cols].sum().apply(lambda x:x/x.sum(), axis='columns')
df_valid_Z = qwe.loc[qwe['train_Z']].groupby(['sex', 'age_2011', 'geo_code'], observed=True)[religion_cols].sum().apply(lambda x:x/x.sum(), axis='columns')

assert df_train_X.index.equals(df_train_Y.index)
assert df_valid_Y.index.equals(df_valid_Z.index)

# Fit models

In [18]:
estimators = [
    LinearTrendEstimator,
    ExponentialEstimator,
    # OddsMultiplierEstimator,
    # TransitionMatrixEstimator,
    BasicTransitionMatrixEstimator,
    IndividualLinearEstimator,
    IndividualExponentialEstimator,
    # IndividualOddsRatioEstimator,
    IndividualBasicTransitionMatrixEstimator,
]

In [19]:
preds = []
estimator_names = []

for est in estimators:
    estimator_names.append(est.__name__)
    preds.append(est().fit(df_train_X, df_train_Y).predict(df_valid_Y).values.flatten())

In [20]:
preds = np.stack(preds, axis=-1)
preds.shape

(47520, 6)

In [21]:
actuals = df_valid_Z.values.flatten()
actuals.shape

(47520,)

## Build an ensemble which minimises error on the validation set

In [22]:
# Function to minimise MSE
def mse_loss(weights):
    weighted_preds = np.dot(preds, weights)
    return mean_squared_error(actuals, weighted_preds)

# Constraints: weights must sum to 1
constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}
bounds = [(0, 1)] * preds.shape[1]  # Each weight between 0 and 1
initial_weights = np.ones(preds.shape[1]) / preds.shape[1]  # Equal starting weights

# Optimisation
result = minimize(mse_loss, initial_weights, bounds=bounds, constraints=constraints)
optimal_weights = result.x

# Compute final weighted predictions
ensemble_preds_weighted = np.dot(preds, optimal_weights)

In [23]:
optimal_weights = pd.Series(optimal_weights, index=estimator_names).rename('Model Weight').to_frame()
optimal_weights.index = optimal_weights.index.rename('Estimator')

In [24]:
optimal_weights

,Model Weight
Estimator,
LinearTrendEstimator,0.033400
ExponentialEstimator,0.107208
BasicTransitionMatrixEstimator,0.000000
IndividualLinearEstimator,0.290923
IndividualExponentialEstimator,0.355209
IndividualBasicTransitionMatrixEstimator,0.213260


In [25]:
optimal_weights.to_csv('optimal_weights_for_over_10_model.csv')

# Make predictions on test set for over tens

## Build datasets

In [26]:
df_test_fit_X = qwe.loc[qwe['test_X']].groupby(['sex', 'age_2001', 'geo_code'], observed=True)[religion_cols].sum().apply(lambda x:x/x.sum(), axis='columns')
df_test_fit_Y = qwe.loc[qwe['test_Y']].groupby(['sex', 'age_2001', 'geo_code'], observed=True)[religion_cols].sum().apply(lambda x:x/x.sum(), axis='columns')
df_test_predict_Y = qwe.loc[qwe['test_Y']].groupby(['sex', 'age_2011', 'geo_code'], observed=True)[religion_cols].sum().apply(lambda x:x/x.sum(), axis='columns')
df_test_predict_Z = qwe.loc[qwe['test_Z']].groupby(['sex', 'age_2011', 'geo_code'], observed=True)[religion_cols].sum().apply(lambda x:x/x.sum(), axis='columns')

assert df_test_fit_X.index.equals(df_test_fit_Y.index)
assert df_test_predict_Y.index.equals(df_test_predict_Z.index)

## Make predictions

In [27]:
test_preds = []

for est in estimators:
    test_preds.append(est().fit(df_test_fit_X, df_test_fit_Y).predict(df_test_predict_Y).values.flatten())

In [28]:
test_preds = np.stack(test_preds, axis=-1)
test_actuals = df_test_predict_Z.values.flatten()
test_preds_weighted = np.dot(test_preds, optimal_weights)

### Evaluate local predictions

In [29]:
print('Root Mean Squared Error:', f'{root_mean_squared_error(test_actuals, test_preds_weighted):.2%}')
print('Mean Absolute Error:', f'{mean_absolute_error(test_actuals, test_preds_weighted):.2%}')

Root Mean Squared Error: 2.09%
Mean Absolute Error: 1.06%


### Evaluate national bias of predictions

In [30]:
(pd.DataFrame(
    test_preds_weighted.reshape(-1, 9),
    df_test_predict_Y.index,
    df_test_predict_Y.columns,
) - df_test_predict_Z).mean().rename('Mean Error').to_frame().style.format('{:.2%}')

,Mean Error
religion,
Buddhist,0.08%
Christian,0.40%
Hindu,-0.01%
Jewish,-0.07%
Muslim,-0.01%
No religion,-1.23%
Not answered,0.96%
Other religion,-0.04%
Sikh,-0.08%


# Under tens
 - I need four models: (sex x 2) * (age_band x 2)
 - Try a variety of predictor datasets including 2011 actuals and 2021 predictions
 - Try every model type
 - Pick the best according to rmse

In [31]:
u10_child_age_bands = ['0-4', '5-9']
u10_parent_age_bands = ['10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40-44']
u10_age_bands = u10_child_age_bands + u10_parent_age_bands

## Create training and validation datasets

In [32]:
u10_train_X = {}
u10_train_Y = {}
u10_valid_Z = {}

for ab in u10_parent_age_bands:
    u10_train_X[ab] = qwe.loc[(qwe['train_X']) & (qwe['age_2001'] == ab)].groupby(['sex', 'geo_code'], observed=True)[religion_cols].sum().apply(lambda x:x/x.sum(), axis='columns')

for ab in u10_age_bands:
    u10_train_Y[ab] = qwe.loc[(qwe['train_Y']) & (qwe['age_2011'] == ab)].groupby(['sex', 'geo_code'], observed=True)[religion_cols].sum().apply(lambda x:x/x.sum(), axis='columns')

for ab in u10_child_age_bands:
    u10_valid_Z[ab] = qwe.loc[(qwe['train_Z']) & (qwe['age_2021'] == ab)].groupby(['sex', 'geo_code'], observed=True)[religion_cols].sum().apply(lambda x:x/x.sum(), axis='columns')

## Train models

In [33]:
u10_models = {}

for sex_of_child in ['female', 'male']:
    for age_of_child in u10_child_age_bands:
        for sex_of_parent in ['female', 'male']:
            for age_of_parent in u10_parent_age_bands:
                for est_class in estimators:
                    
                    # direct
                    key = ('direct', sex_of_child, age_of_child, sex_of_parent, age_of_parent, est_class.__name__)
                    u10_models[key] = est_class().fit(u10_train_X[age_of_parent].loc[sex_of_parent], u10_train_Y[age_of_child].loc[sex_of_child])

                    # from over-10 predictions
                    # if age_of_parent not in ['15-19', '20-24']:
                    key = ('via_preds', *key[1:])
                    u10_models[key] = est_class().fit(u10_train_Y[age_of_parent].loc[sex_of_parent], u10_train_Y[age_of_child].loc[sex_of_child])

In [34]:
len(u10_models)

672

## Make predictions and validate them

### Use the predictions of parent religion from the over ten ensemble model

In [35]:
ages_2011 = ['0-4','5-9',
             '10-14','15-19','20-24','25-29','30-34','35-39','40-44',
             '45-49','50-54','55-59','60-64','65-69','70-74']

age_2021_from_age_2011 = {a: f'{int(a.split("-")[0]) + 10}-{int(a.split("-")[1]) + 10}' for a in ages_2011}
age_2021_from_age_2011['75+'] = '85+'

In [36]:
u10_2021_preds = pd.DataFrame(
    ensemble_preds_weighted.reshape(-1,9),
    index=df_valid_Y.index,
    columns=df_valid_Y.columns
)

u10_2021_preds = u10_2021_preds.reset_index()
u10_2021_preds = u10_2021_preds[u10_2021_preds['age_2011'].isin(u10_parent_age_bands)]

# parents are now 10 years older
u10_2021_preds['age_2021'] = u10_2021_preds['age_2011'].map(age_2021_from_age_2011)
u10_2021_preds = u10_2021_preds.set_index(['sex', 'age_2021', 'geo_code']).drop(columns='age_2011')

### Predict child religion

In [37]:
u10_predictions = {}
u10_rmse = {}

for sex_of_child in ['female', 'male']:
    for age_of_child in u10_child_age_bands:
        for sex_of_parent in ['female', 'male']:
            for age_of_parent in u10_parent_age_bands:
                for est_class in estimators:

                    actuals = u10_valid_Z[age_of_child].loc[sex_of_child]
                    
                    # direct
                    key = ('direct', sex_of_child, age_of_child, sex_of_parent, age_of_parent, est_class.__name__)
                    preds = u10_models[key].predict(u10_train_Y[age_of_parent].loc[sex_of_parent])
                    u10_predictions[key] = preds
                    u10_rmse[key] = root_mean_squared_error(actuals, preds)
        
                    # from over-10 predictions
                    if age_of_parent not in ['10-14', '15-19']:
                        key = ('via_preds', *key[1:])
                        preds = u10_models[key].predict(u10_2021_preds.loc[(sex_of_parent, age_of_parent)])
                        u10_predictions[key] = preds
                        u10_rmse[key] = root_mean_squared_error(actuals, preds)

### Evaluate

In [38]:
results = pd.Series(u10_rmse).reset_index()
results.columns = ['approach', 'sex_of_child', 'age_of_child', 'sex_of_parent', 'age_of_parent', 'estimator', 'rmse']

In [39]:
best = results.sort_values(['sex_of_child', 'age_of_child', 'rmse']).groupby(['sex_of_child', 'age_of_child']).nth(0)
best

,approach,sex_of_child,age_of_child,sex_of_parent,age_of_parent,estimator,rmse
63,via_preds,female,0-4,female,40-44,ExponentialEstimator,0.017442
215,via_preds,female,5-9,female,40-44,IndividualBasicTransitionMatrixEstimator,0.013471
351,via_preds,male,0-4,female,40-44,ExponentialEstimator,0.017065
503,via_preds,male,5-9,female,40-44,IndividualBasicTransitionMatrixEstimator,0.014451


In [40]:
best.to_csv('optimal_under_10_models.csv', index=False)

# Train and predict under ten models on test set

In [41]:
u10_2021_test_preds = pd.DataFrame(
    test_preds_weighted.reshape(-1,9),
    index=df_test_predict_Y.index,
    columns=df_test_predict_Y.columns
).reset_index()

# parents are now 10 years older
u10_2021_test_preds['age_2021'] = u10_2021_test_preds['age_2011'].map(age_2021_from_age_2011)
u10_2021_test_preds = u10_2021_test_preds.set_index(['sex', 'age_2021', 'geo_code']).drop(columns='age_2011')

## Make datasets

In [42]:
u10_test_Y = {}
u10_test_Z = {}

for ab in u10_age_bands:
    u10_test_Y[ab] = qwe.loc[(qwe['test_Y']) & (qwe['age_2011'] == ab)].groupby(['sex', 'geo_code'], observed=True)[religion_cols].sum().apply(lambda x:x/x.sum(), axis='columns')

for ab in u10_child_age_bands:
    u10_test_Z[ab] = qwe.loc[(qwe['test_Z']) & (qwe['age_2021'] == ab)].groupby(['sex', 'geo_code'], observed=True)[religion_cols].sum().apply(lambda x:x/x.sum(), axis='columns')

## Make predictions

In [43]:
test_models = {}
test_predictions = {}
test_errors = {}

cols = ['sex_of_child', 'age_of_child', 'sex_of_parent', 'age_of_parent', 'estimator']
for _, (sex_of_child, age_of_child, sex_of_parent, age_of_parent, estimator) in best[cols].iterrows():
    test_models[(sex_of_child, age_of_child)] = locals()[estimator]().fit(u10_test_Y[age_of_parent].loc[sex_of_parent], u10_test_Y[age_of_child].loc[sex_of_child])
    
    preds = test_models[(sex_of_child, age_of_child)].predict(u10_2021_test_preds.loc[(sex_of_parent, age_of_parent)])
    test_predictions[(sex_of_child, age_of_child)] = preds
    
    actuals = u10_test_Z[age_of_child].loc[sex_of_child]
    test_errors[(sex_of_child, age_of_child)] = preds - actuals

In [44]:
# for k, v in test_errors.items():
#     print(k, (v.values ** 2).mean() ** 0.5),
#     display(v.mean())

## Combine with over 10 predictions

In [45]:
for k in test_predictions:
    test_predictions[k].index = pd.MultiIndex.from_product([[k[0]], [k[1]], test_predictions[k].index])

test_predictions_2021 = pd.concat([*test_predictions.values(), u10_2021_test_preds]).sort_index()
test_actuals_2021 = qwe[qwe['test_Z']].reset_index().groupby(['sex', 'age_2021', 'geo_code'], observed=True)[religion_cols].sum().apply(lambda x: x/x.sum(), axis='columns').sort_index()

assert len(set(test_actuals_2021.index) ^ set(test_predictions_2021.index)) == 0
test_predictions_2021.index.names = test_actuals_2021.index.names

test_errors_2021 = test_predictions_2021 - test_actuals_2021

# Inspect errors

In [46]:
pd.DataFrame({
    'Mean Error (bias)': test_errors_2021.mean(),
    'Root Mean Squared Error': (test_errors_2021 ** 2).mean() ** 0.5,
    'Mean Absolute Error': test_errors_2021.abs().mean(),
    'Median Absolute Error': test_errors_2021.abs().median(),
    '5th percentile': test_errors_2021.quantile(0.05),
    '95th percentile': test_errors_2021.quantile(0.95),
}).style.format('{:.2%}')

,Mean Error (bias),Root Mean Squared Error,Mean Absolute Error,Median Absolute Error,5th percentile,95th percentile
religion,,,,,,
Buddhist,0.08%,0.44%,0.28%,0.19%,-0.45%,0.80%
Christian,0.39%,4.03%,3.08%,2.46%,-5.07%,7.58%
Hindu,0.03%,0.85%,0.45%,0.27%,-0.74%,1.22%
Jewish,-0.07%,0.48%,0.20%,0.12%,-0.39%,0.29%
Muslim,0.06%,1.59%,0.78%,0.36%,-1.37%,2.42%
No religion,-1.18%,3.26%,2.37%,1.73%,-6.71%,3.35%
Not answered,0.81%,3.02%,1.93%,1.45%,-2.59%,4.21%
Other religion,-0.05%,0.51%,0.30%,0.19%,-0.66%,0.56%
Sikh,-0.07%,0.37%,0.23%,0.17%,-0.44%,0.45%


In [47]:
test_errors_2021.groupby('age_2021').mean().style.format('{:.2%}')

religion,Buddhist,Christian,Hindu,Jewish,Muslim,No religion,Not answered,Other religion,Sikh
age_2021,,,,,,,,,
0-4,0.11%,-0.44%,0.36%,-0.15%,0.96%,-0.25%,-0.40%,-0.12%,-0.06%
10-14,0.08%,1.39%,0.00%,-0.04%,-0.03%,-0.88%,-0.34%,-0.08%,-0.11%
15-19,0.09%,3.43%,0.02%,-0.04%,0.10%,-1.76%,-1.59%,-0.17%,-0.08%
20-24,0.15%,3.95%,0.04%,-0.04%,0.43%,-2.85%,-1.32%,-0.24%,-0.12%
25-29,0.21%,5.27%,0.11%,-0.05%,0.32%,-5.10%,-0.44%,-0.23%,-0.09%
30-34,0.26%,3.65%,0.26%,-0.05%,0.11%,-3.47%,-0.50%,-0.16%,-0.11%
35-39,0.22%,1.41%,-0.04%,-0.07%,-0.02%,-0.90%,-0.40%,-0.08%,-0.12%
40-44,0.15%,-0.65%,-0.08%,-0.08%,-0.27%,0.96%,0.13%,-0.02%,-0.14%
45-49,0.09%,-1.37%,-0.02%,-0.06%,-0.16%,0.72%,0.90%,0.03%,-0.12%
